In [20]:
using JuMP, Plots, Cbc, GraphRecipes, LightGraphs, Distributions, SimpleWeightedGraphs, SparseArrays, CSV, Random, DataFrames, JLD, Geodesy

In [1]:
# import Pkg;
# using Pkg
# Pkg.add("Cbc");
# Pkg.add("GraphRecipes");
# Pkg.add("LightGraphs");
# Pkg.add("SimpleWeightedGraphs");
# Pkg.add("SparseArrays");
# Pkg.add("CSV");
# Pkg.add("DataFrames");
# Pkg.add("Geodesy");
# Pkg.add("JLD")
# Pkg.add("Random")

### ECE/CS/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2022 ###


 Madison Bus Route Designing: Preliminary Report
=====

Team Members: Rabeeya Hamid, Ai Guan, Naman Gupta

Data: 04/22/2022

*****

### Table of Contents

1. [Problem Description](#1.-Problem-Description)
2. [Data Collection](#2.-Data-Collection)
3. [Mathematical Model](#3.-Mathematical-Model)
4. [Preliminary Code Implementation](#4.-Code-Implementation)
5. [Preliminary Results and Analysis](#5.-Results-and-Analysis)


# 1. Problem Description


Based on the increasing need of public transportation, the City of Madison Metro Transit is proposing a major network redesign ([Transit Network Redesign](https://www.cityofmadison.com/metro/routes-schedules/transit-network-redesign)). Currently, they are surveying the students of UW-Madison and the residents of Madison for suggestions on their travel preferences, so that they can re-design the bus routes to meet the ridership demand and businesses. As University of Wisconsin-Madison students, our team thinks the redesign project would also have significant impact on our daily lives. 

The aim of the project is to model the routing design as a network optimization problem, so that we can propose the model as a part of their feedback survey. The family of network optimization problem includes assignment, critical path, max flow, shortest path, transportation, and min-cost flow problems. One of the primary results of the model is a network of arcs and nodes that illustrate the optimal routes for the nodes in a network ([Network Optimization](http://home.ubalt.edu/ntsbarsh/opre640a/partIII.htm#:~:text=The%20family%20of%20network%20optimization,called%20a%20vertex%2C%20or%20point)).

The project is a transportation problem in the network optimization problem family, and the nodes represent the bus stops, and the arcs represent the transportation flow between two stations. The model consist of three parts: decision variables, constraints, and objective function. We would like to model the optimal routes to meet the demand of ridership for each bus stop in Madison. Therefore, the objective function of the problem is to maximize the number of passengers boarding each bus on all the bus stops in Madison,  subject to fuel limitation of each bus, the capacity of a bus, routing regulations and ridership demand for each bus stop. 

-----

# 2. Data Collection

The data utilized in the project is the publicly available data from the City of Madison website. Two datasets are selected corresponding to the 
   1.	Metro Transit Ridership by Stop | City of Madison Open Data.  [Data Source](https://data-cityofmadison.opendata.arcgis.com/datasets/cityofmadison::metro-transit-ridership-by-stop)
   2.	Metro Transit Bus Route Patterns | City of Madison Open Data. [Data Source](https://data-cityofmadison.opendata.arcgis.com/datasets/cityofmadison::metro-transit-bus-route-patterns/about)
   3.   Metro Transit Bus Stops | City of Madison Open Data. [Data Source](https://data-cityofmadison.opendata.arcgis.com/maps/metro-transit-bus-stops)
    4. Metro Transit Trips | City of Madison Open Data. [Data Source](https://data-cityofmadison.opendata.arcgis.com/datasets/cityofmadison::metro-transit-bus-route-trips)

* The first dataset contains a list of all the bus stops, their geographical location (longitude and latitude) and the ridership at that bus stop averaged over 12 days. We do not have distances between these bus stops, nor do we have connections between them, for which we propose the following approach. 
   *   We calculate the Geodesic Euclidean distance between the geographic position of the bus stops to define the distances between them. 
   *   These distances will be multiplied by a constant of proportionality to determine the fuel cost associated with each path. 

* The second dataset contains a list of present bus routes, the bus stops along the routes and the total ridership for the bus routes averaged over 12 days. 
    
* The third dataset maps the bus stops to the bus routes taken by all the buses in Madison. 
* The fourth dataset maps the bus routes to the bus stops 

Further, we load all the data and calculate the euclidean distance between each bus stop in Madison. We give a brief demo - 

## Importing datasets:
Importing all four data sets as dataframes.

In [21]:
# import data set as a dataframe

# 1
ridership_data = CSV.read("data/Metro_Transit_Ridership_by_Stop.csv", DataFrame)
# 2
route_patterns_data = CSV.read("data/Metro_Transit_Bus_Route_Patterns.csv", DataFrame)
# 3
stops_data = CSV.read("data/Metro_Transit_Bus_Stops.csv", DataFrame)
# 4
weekday_data = CSV.read("data/Metro_Transit_Ridership_by_Route_Weekday.csv", DataFrame)
;

## Calculating distances between all bus stops:

The distance between each bus stop in Madison as a $n * n$ matrix.

We map the bus stops (ids) to bus routes  that will help us model the optimal bus routes.

### Creating a vector of bus stops within routes:

In [22]:
# bus_route = [80, 15, 4, 2, 7, 10, 17]
route_stops = []

for i in eachrow(stops_data)
     if i.Route != "None"
        push!(route_stops, i.stop_id)
    end 
end

print("Number of bus stops: ", length(route_stops))

Number of bus stops: 1744

### Storing locations and calculating Euclidian distances between all nodes:

In [23]:
stops = []

stop_locations = Vector{LatLon{Float64}}()
intersections = []
stop_ids = []

for i in eachrow(ridership_data)
    int = i.IntersectionID
    if i.StopID in route_stops
        if int ∉ intersections
            push!(intersections, int)
            x = LatLon(i.Lat, i.Lon)
            push!(stop_locations, x)
            push!(stop_ids, i.StopID)
        end
    end
end

N = length(stop_ids)

distances = zeros(N,N)

for i in 1:N
    for j in 1:N
        distances[i,j] = euclidean_distance(stop_locations[i], stop_locations[j])
    end
end

distances
stop_locations

lat = [x.lat for x in stop_locations][1:N-2]
long = [x.lon for x in stop_locations][1:N-2]
;

## Creating ridership data:

In [24]:
ridership = Dict()

for i in eachrow(weekday_data)
    if i.StopID in stop_ids
        ridership[i.StopID] = i.DailyBoardings
    end
end
r = collect(values(ridership))

remove =  N - length(r)

n = N - remove
print("Number of bus stops in data: ", n)

Number of bus stops in data: 799

### Assigning distances matrix:

In [25]:
D = distances[1:n,1:n]
;

## Creating adjacency matrix: 

Naman: Update description here. <br>
Creating $n*n$ Adjacency Matrix on the basis of StopIDs. We assume a bus stop with StopID 5 is connected to StopID 6.

### Loading adjacency matrix:

In [ ]:
load("Adj.jld")["Adj Matrix"]
;
### do not run the next cell

### Code for adding (random direction) edges between close bus stops:

We allow our buses to travel to close (distance<500) bus stops, but don't allow it to travel to bus stops that are far away (d>1000). For choosing the direction when adding an edge, we use a discrete random variable between 0 and 1. Which is also why we have commented out this code and saved the ```Adj``` matrix, since running it again would change the matrix. 

In [19]:
# Adj = zeros(N,N)

# index_stopid = Dict()

# for row in eachrow(ridership_data)
#     if row.StopID in route_stops
#         i = rownumber(row)
#         index_stopid[i] = row.StopID
#     end
# end
# index_stopid
# for i in 1:N
#     if i + 1 < N
#         Adj[i, i + 1] = 1
#         # Adj[i + 1, i] = 1
#     end
# end


# for i in 1:N
#     for j in 1:N
#         if i != j
#         d = distances[i,j]
#             if d > 1000
#                 Adj[i, j] = 0
#                 Adj[j, i] = 0
#             end
#                 if d < 500
#                     p = rand((0,1))
#                     Adj[i, j] = p
#                     Adj[j, i] = 1-p
#                 end
#         end
#     end
# end

# Adj = Adj[1:n,1:n]
# ;

### Converting adjacency matrix into a directed graph:

In [28]:
using Graphs

dirGraph = Graphs.DiGraph(Adj) # converting adjacency matrix into a directed graph

{799, 1789} directed simple Int64 graph

### Removing cycles from the Adjacency Matrix:

In [ ]:
# cycles = Graphs.simplecycles(dirGraph)
# for i in 1:length(cycles)
#     Adj[cycles[i][1], cycles[i][2]] = 0
# end

# dirGraph = Graphs.DiGraph(Adj)
# Adj

### Saving Adjacency Matrix:

In [26]:
save("Adj.jld", "Adj Matrix", Adj)
;

----


# 3. Mathematical Model


### Parameters Given:
##### From Data:
- Number of bus stops ```n``` : number of nodes in our network
- Adjacency Matrix ```Adj``` ($n \times n$) : describes connections between bus stops
- Distance Matrix ```D``` ($n \times n$): Euclidean distances between all bus stops
- Ridership vector ```r```($n \times 1$): the ridership demand at every bus stop 


##### Assigned ourselves: 
- Number of buses ```num_buses```: the total number of buses running in our model
- Start node ```n_i```($1 \times \text{num_buses}$): the bus stops from where our buses start, unique for every bus.
- End node ```n_e```($1 \times \text{num_buses}$): the bus stops where our bus routes end, unique for every bus.
- Distance (Fuel) limit ```f```($1 \times \text{num_buses}$): the distances each of our buses can travel in total/the fuel that each bus is allowed to use. Unique for every bus.

## Loading ```Adj``` matrix:

In [27]:
load("Adj.jld")["Adj Matrix"]
;

### Assigning number of buses:

In [ ]:
num_buses = 3


## Assigning ```b``` vector using start and end nodes:

In [1028]:
using Random

# n_i = rand(1:n,3,1)
# n_e = rand(1:n,3,1)

### hard coding start and end nodes for now:

n_i = [3, 4, 12]
n_e = [92,10, 13]

b = zeros(n, num_buses)
for i in 1:num_buses
    b[n_i[i], i] = -1
    b[n_e[i], i] = 1
end
b = Int.(b)
;

"start2end.csv"

### Storing start and end nodes in a CSV file:

In [ ]:
start_end = hcat(n_i, n_e) 

CSV.write("start2end.csv",  DataFrame(start_end, :auto), writeheader=["start_node", "end_node"])

## Defining fuel vector ```f```:

In [ ]:
f = [1e10, 1e10, 1e10]
;

### Parameters to be calculated:
- Number of paths ```e```: number of edges in our network 
- Incidence matrix ```Inc```($n \times e$): describes the edges in our network.
- Edge costs ```d_e```($e \times 1$): the distance corresponding to edges in the network
- Rider intake ```r_e```($e \times 1$): the riders taken up after traveling to each edge
- Flow vector ```b``` ($n \times \text{num_buses}$): includes the start and end node

## Calculating incidence matrix ```Inc``` and number of edges ```e```:

In [989]:
Inc = Array(Graphs.incidence_matrix(dirGraph))
# Inc = Array(my_incidence_matrix(dirGraph)) # converting directed graph to an incidence matrix

e = size(Inc)[2] # number of edges

837

## Defining edge costs ```d_e``` and rider intake ```r_e```:

In [1032]:

temp_D = D.*Adj
temp_r = zeros((n,n))

for i = 1:n
        temp_r[i, :] = r.*Adj[i, :]
end

d_e = zeros(0)
r_e = zeros(0)

for i = 1:n
    for j = 1:n
        if temp_D[i, j] != 0
            append!(d_e, temp_D[i, j])
        end
        if temp_r[i, j] != 0
            append!(r_e, temp_r[i, j])
        end
    end    
end

## Final Mathematical Model:


**Decision variable:** <br>

$X$: all possible edges in the network, for each bus. Shape = $e \times \text{num_buses}$ <br>
    $$ X[e_i, k_i] =   \left\{
\begin{array}{ll}
      1 & \text{If the bus $k_i$ travels on edge $e_i$}  \\
      0 & \text{If the bus $k_i$ does NOT travel on edge $e_i$} \\
\end{array} 
\right.  $$

**Objective function:** Maximize people getting on the bus, for all buses <br>
$$ \max_{x} \Sigma_{k = 1}^{\text{num_buses}}\text{    }r_e \cdot X[:, k]$$

**Constraints:** <br>
  s.t. <br>
For every bus in ```num_buses``` (```X[:, k] = x```): <br> <br>
Nodal conservation:
$$ \text{Inc}*x = b, $$
Fuel Constraint:
$$ d_e \cdot x <= f  $$
Bus can only leave a bus stop once:
$$ \Sigma_{j} x_ij \leq 1\forall i \in [1, n] $$ 
Bus can only enter a bus stop once: 
$$ \Sigma_{i} x_ij \leq 1\forall j \in [1, n] $$

For all buses:
$$X[:, k_1] \cdot X[:, k_2]  \cdot..... \cdot X[:,k_{end}] = 0$$
Bus routes can not overlap with other buses. 

----

# 4. Code Implementation

In [1034]:
using LinearAlgebra, Ipopt, Gurobi
bus_route = Model(with_optimizer(Gurobi.Optimizer,OutputFlag=0, NonConvex = 2))

@variable(bus_route, X[1:e, 1:num_buses], Bin) # binary decision variable x that maps out the [path]

for i in 1:num_buses
    @constraint(bus_route, Inc*X[:, i] .== b[:, i]) # nodal conservation
    @constraint(bus_route, d_e'*X[:, i] <= f[i]) # fuel constraint
    @constraint(bus_route, (floor.((Inc.+1)./2))*X[:, i] .<= 1) # can leave from one node once only
    @constraint(bus_route, (-1*ceil.((Inc.-1)./2))*X[:, i] .<= 1) # can enter one node once only
end


for i in 1:num_buses
    for j in 1:num_buses
        if i != j
            @constraint(bus_route, dot(X[:, i],X[:, j]) <= 0.9) # not share path
        end
    end
end

@objective(bus_route, Max, sum(r_e'*X[:, i] for i in 1:num_buses))  # maximize people getting on the bus
    
s = optimize!(bus_route)

termination_status(bus_route)
#print(bus_route)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-15
Set parameter NonConvex to value 2


OPTIMAL::TerminationStatusCode = 1

----

# 5. Results and Analysis:

## Defining dictionary for converting edge index to path:
To make it easier to map the optimal decision variable to the path it takes. 

In [1035]:
edgeIndex2path = Dict()
count = 1
for i = 1:n
    for j = 1:n
        if Adj[i, j] != 0
            edgeIndex2path[count] = (i, j)
            count = count + 1
        end
    end
end

sorted_edges = sort(collect(pairs(edgeIndex2path)), by=x->x[1])

for i in sorted_edges
    println(i)
end

Pair{Any, Any}(1, (3, 2))
Pair{Any, Any}(2, (3, 4))
Pair{Any, Any}(3, (3, 51))
Pair{Any, Any}(4, (3, 82))
Pair{Any, Any}(5, (3, 92))
Pair{Any, Any}(6, (4, 5))
Pair{Any, Any}(7, (4, 10))
Pair{Any, Any}(8, (4, 89))
Pair{Any, Any}(9, (4, 92))
Pair{Any, Any}(10, (5, 6))
Pair{Any, Any}(11, (5, 13))
Pair{Any, Any}(12, (7, 8))
Pair{Any, Any}(13, (7, 69))
Pair{Any, Any}(14, (7, 84))
Pair{Any, Any}(15, (8, 2))
Pair{Any, Any}(16, (9, 1))
Pair{Any, Any}(17, (9, 10))
Pair{Any, Any}(18, (9, 45))
Pair{Any, Any}(19, (9, 62))
Pair{Any, Any}(20, (10, 11))
Pair{Any, Any}(21, (12, 5))
Pair{Any, Any}(22, (12, 13))
Pair{Any, Any}(23, (12, 50))
Pair{Any, Any}(24, (12, 77))
Pair{Any, Any}(25, (13, 14))
Pair{Any, Any}(26, (15, 16))
Pair{Any, Any}(27, (15, 22))
Pair{Any, Any}(28, (15, 35))
Pair{Any, Any}(29, (15, 37))
Pair{Any, Any}(30, (16, 17))
Pair{Any, Any}(31, (16, 20))
Pair{Any, Any}(32, (17, 29))
Pair{Any, Any}(33, (17, 42))
Pair{Any, Any}(34, (18, 58))
Pair{Any, Any}(35, (19, 20))
Pair{Any, Any}(36, (1

## Defining function for finding the path taken:
Takes as input our decison variable and gives as output a dictionary which has all possible edges as keys and 0s and 1s as values. Where 0 signifies that our bus did not travel on that edge, and 1 signifies that it did travel on that edge.

In [1036]:
function path_taken(opt_path)
    # finding edges travelled using edgeIndex2path dictionary
    path_taken_x = zeros(0)
    path_taken_y = zeros(0)
    for i = 1:e
        if opt_path[i] != 0
            temp = edgeIndex2path[i]
            append!(path_taken_x, temp[1] )
            append!(path_taken_y, temp[2] )
        end
    end
    travelled_edges_dict = Dict()
    # defining dictionary with 1 for travelled edges, 0 for untravelled edges
    for k = 1:length(path_taken_x)
        for i in 1:n
            for j in 1:n
                if i == path_taken_x[k] && j == path_taken_y[k] 
                    travelled_edges_dict[(i, j)] = 1
                elseif haskey(travelled_edges_dict, (i, j))
                    if travelled_edges_dict[(i, j)] == 1
                        travelled_edges_dict[(i, j)] = 1
                    end
                else 
                    travelled_edges_dict[(i, j)] = 0
                end
            end
        end
    end

    # printing the edges travelled on

    print("The bus travelled on these edges: \n")
    for i = 1:n
        for j = 1:n
            if travelled_edges_dict[(i, j)] != 0
                println((i, j))
            end
        end
    end
return travelled_edges_dict
end

path_taken (generic function with 1 method)

## Printing paths travelled for every bus:

In [1037]:
opt_path = value.(X)
num_buses = 3
dicts = []

for i in 1:num_buses
    dict1 = path_taken(opt_path[:, i])
    push!(dicts, dict1)
end

The bus travelled on these edges: 
(3, 51)
(51, 92)
The bus travelled on these edges: 
(4, 10)
The bus travelled on these edges: 
(3, 4)
(4, 5)
(5, 13)
(12, 77)
(77, 3)


## Defining function for finding nodes travelled to: NOT USING


In [ ]:
function nodes_travelled(opt_path, bus_no)
    nodes = 1:n
    opt_path_bus = opt_path[:, bus_no]
    temp_path = zeros((n,n))

    # defining temporary variable temp_path to store nodes travelled to for on each edge index
    # (zeros need to be filtered out) 
    for i = 1:n
            temp_path[i, :] = nodes.*Adj[i, :]
    end

    path = zeros(0)

    # filtering out zeros
    for i = 1:n
        for j = 1:n
            if temp_path[i, j] != 0
                append!(path, temp_path[i, j])
            end
        end    
    end


    # finding actual nodes travelled to using optimal edge vector x
    temp_path = path.*opt_path_bus


    path = zeros(0)

    # filtering out zeros again
    for p in temp_path
        if p != 0
            append!(path, p)
        end
    end

    # defining membership based on nodes travelled to, 1 for not travelled to, 2 for travelled to
    for i = 1:n
        for p in path 
            if round(Int, p) == i
                membership[i] = bus_no + 1
            end
        end
    end

    # start node is also travelled to 
    membership[n_i[bus_no]] = bus_no + 1
end

## Exporting data for visualization in python:
Export longitude lattitude for the connections between bus stops (total and optimal), and the longitude lattitude of the bus stops as well. 

In [ ]:
sources = Dict() # Stores connections 

all_sources = [] ## vector for all possible connections
for i in sorted_edges
    source = i[2][1]
    sink = i[2][2]
    x = lat[source], long[source], lat[sink], long[sink], source, sink
    
    push!(all_sources, x)
    
end

sources[1] = all_sources # append to sources

# for each dictionary, create a matrix which stores the long and lat of nodes travelled on

idx = 2
for dict in dicts   
    loc = []
    for (key, value) in dict
        if value != 0
            i = key
            source = i[1]
            sink = i[2]
            x = lat[source], long[source], lat[sink], long[sink], source, sink
            push!(loc, x)
        end
    end
    sources[idx] = loc
    idx +=1
end


# store all four as seperate CSV files:
A = sources
for (k,v) in A
    CSV.write("Connections"* string(k)*".csv",  DataFrame(v), writeheader=["start_lat", "start_lon", "end_lat", "end_lon", "start_node", "end_node"])
end



# store longitude and lattitude of all bus stops

count = 1
busstops = []
for x in stop_locations
    push!(busstops, (count, x.lat, x.lon))
    count+=1
end

B = busstops[1:N-2]

CSV.write("BusStops.csv", DataFrame(B), writeheader=["node", "lon", "lat"])

## Bus route visualization:

Include figure from python file

## Further stats:

like fuel used, people picked up, number of nodes travelled etc

In [ ]:
# println("Fuel used: ", dot(d_e, opt_path))
# println("People picked up: ", dot(r_e, opt_path) + r[n_i]) 

-----

# 6. Further Work: